In [ ]:
%reload_ext autoreload
%autoreload 2

In [1]:
# 01_data_exploration.ipynb

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from src.data.downloader import MarketDataDownloader
from src.data.processor import LossAversionProcessor

# Initialize downloader
downloader = MarketDataDownloader(
    data_dir='data',
    start_date='2020-01-01'
)

# Get S&P 500 tickers and download data
sp500_tickers = downloader.download_sp500_constituents()
stock_data = downloader.download_stock_data(sp500_tickers)

# Process first 50 stocks
processor = LossAversionProcessor(data_dir='data')
event_data = []

for ticker in list(stock_data.keys())[:50]:
    df, metrics = processor.process_stock(ticker)
    event_data.append(df)

event_df = pd.concat(event_data)

# Basic statistics
print("Event Statistics:")
print(event_df.groupby('Event_Type').agg({
    'Abnormal_Volume': ['mean', 'std', 'count'],
    'Abnormal_Return': ['mean', 'std']
}))

# Volume response visualization
plt.figure(figsize=(10, 6))
sns.boxplot(data=event_df, x='Event_Type', y='Abnormal_Volume')
plt.title('Volume Response by Event Type')
plt.show()

# Save processed data
event_df.to_parquet('data/processed/exploration_results.parquet')

# 02_loss_aversion_analysis.ipynb

import numpy as np
import pandas as pd
from src.analysis.loss_aversion import LossAversionAnalyzer
from src.analysis.statistics import StatisticalTests

# Initialize analyzer
analyzer = LossAversionAnalyzer(
    data_dir='data',
    output_dir='results'
)

# Load processed data
event_data = pd.read_parquet('data/processed/exploration_results.parquet')

# Run analysis
volume_response = analyzer.analyze_volume_response(event_data)
price_momentum = analyzer.analyze_price_momentum(event_data)
magnitude_effects = analyzer.analyze_magnitude_effects(event_data)

# Statistical testing
stats = StatisticalTests()
gain_events = event_data[event_data['Event_Type'] == 'Gain_Event']
loss_events = event_data[event_data['Event_Type'] == 'Loss_Event']

volume_stats = stats.compare_means(
    gain_events['Abnormal_Volume'].values,
    loss_events['Abnormal_Volume'].values
)

# Save results
results = {
    'volume_response': volume_response,
    'price_momentum': price_momentum,
    'magnitude_effects': magnitude_effects,
    'statistical_tests': volume_stats
}

pd.to_pickle(results, 'results/analysis_results.pkl')
